# Read the data ###

In [1]:
# Let's read in our document-term matrix
import pandas as pd
import pickle

In [2]:
data = pd.read_csv('data/graph_test.csv') #dataset with title
data = data.sort_values(by='modularity_class')
data = data.drop(columns=['timeset'])

data_e = pd.read_csv('data/extracts.csv') #dataset with scraped extracts (summary of page)from web 
data_t = pd.read_csv('data/texts.csv') #dataset with scraped extracts (texts of page)from web

df = data.join(data_e.set_index('id'), on='Id')
df = df.join(data_t.set_index('id'), on='Id')
df = df.dropna()
df = df.drop(columns=['Label'])

In [3]:
df.head()

,Id,modularity_class,title,extract,text
176,427412,0,Saint-Nazaire,Saint-Nazaire (French pronunciation: ​[sɛ̃.na....,"<p class=""mw-empty-elt"">\n</p>\n<p><b>Saint-Na..."
396,152639,0,"Robert Stewart, Viscount Castlereagh","Robert Stewart, 2nd Marquess of Londonderry, ...","<p class=""mw-empty-elt"">\n\n</p>\n<p><b>Robert..."
541,46383,0,Jane Seymour,Jane Seymour (c. 1508 – 24 October 1537) was Q...,"<p class=""mw-empty-elt"">\n</p>\n\n<p><b>Jane S..."
131,1057856,0,Jahangir Khan,"Jahangir Khan, HI (Pashto / Urdu: جهانگير خان‎...","<p class=""mw-empty-elt"">\n\n</p>\n<p><b>Jahang..."
428,75795,0,Juan Carlos I of Spain,Juan Carlos I (Spanish: [xwaŋˈkaɾlos]; Juan Ca...,"<p class=""mw-empty-elt"">\n</p>\n\n<p><b>Juan C..."


In [4]:
def chunks(l, n):
    n = max(1, n)
    return (l[i:i+n] for i in range(0, len(l), n))

## Complete the dataset - web scraping ###

In [ ]:
# import requests
# import json

# df = pd.DataFrame(columns=['id', 'title', 'extract'])

# ids = sorted(data.Id.tolist())
# id_chunks = chunks(ids, 49)
# for chunk in id_chunks:
#     string_ids = ""
#     for idd in chunk:
#         string_ids = string_ids + str(idd) + '|'
#     base_url = 'https://en.wikipedia.org/w/api.php?format=json&action=query&prop=extracts&exintro&explaintext&redirects=1&pageids='
#     url = base_url + string_ids[:-1]
#     res = requests.get(url)
#     response = json.loads(res.text)
#     print(response)
#     for key in response['query']['pages']:
#         try:
#             pageid = response['query']['pages'][key]['pageid']
#             title = response['query']['pages'][key]['title']
#             extract = response['query']['pages'][key]['extract']
#             df = df.append({'id': pageid, 'title': title, 'extract': extract}, ignore_index=True)
#         except KeyError as e:
#             pass
# #             print(e, key)

In [83]:
# import requests
# import json

# df = pd.DataFrame(columns=['id', 'title', 'extract'])

# ids = sorted(data.Id.tolist())
# for idd in ids:
#     base_url = 'https://en.wikipedia.org/w/api.php?format=json&action=query&prop=extracts&exintro&explaintext&redirects=1&pageids='
#     url = base_url + str(idd)
#     res = requests.get(url)
#     response = json.loads(res.text)
#     for key in response['query']['pages']:
#         try:
#             pageid = response['query']['pages'][key]['pageid']
#             title = response['query']['pages'][key]['title']
#             extract = response['query']['pages'][key]['extract']
#             df = df.append({'id': pageid, 'title': title, 'extract': extract}, ignore_index=True)
#         except KeyError as e:
#             print(e, key)

In [82]:
# import requests
# import json

# df = pd.DataFrame(columns=['id', 'text'])

# ids = sorted(data.Id.tolist())
# for idd in ids:
#     base_url = 'https://en.wikipedia.org/w/api.php?format=json&action=query&prop=extracts&redirects=true&pageids='
#     #base_url = 'https://en.wikipedia.org/w/api.php?format=json&action=query&prop=extracts&exintro&explaintext&redirects=1&pageids='
#     url = base_url + str(idd)
#     res = requests.get(url)
#     response = json.loads(res.text)
#     for key in response['query']['pages']:
#         try:
#             pageid = response['query']['pages'][key]['pageid']
#             texts = response['query']['pages'][key]['extract']
#             df = df.append({'id': pageid, 'text': texts}, ignore_index=True)
#         except KeyError as e:
#             print(e, key)

In [6]:
grouped_title = df.groupby("modularity_class")['title'].apply(' '.join).reset_index()

In [7]:
grouped_extract = df.groupby("modularity_class")['extract'].apply(' '.join).reset_index()

In [8]:
grouped_extract.head()

,modularity_class,extract
0,0,Saint-Nazaire (French pronunciation: ​[sɛ̃.na....
1,1,Gadsby is a 1939 novel by Ernest Vincent Wrigh...
2,2,Bombshell is a 1933 American pre-Code romantic...
3,3,"Patrick Michael Shanahan (born June 27, 1962) ..."
4,4,West Side Story is a 1961 American romantic mu...


# Clean the data

In [9]:
import re
import string

# Apply a first round of text cleaning techniques
def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.strip()
    text = text.lower()
    text = text.replace('\n', ' ')
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub(r'[^\x00-\x7F\x80-\xFF\u0100-\u017F\u0180-\u024F\u1E00-\u1EFF]', u'', text)  #remove non-latin chars 
    return text

round1 = lambda x: clean_text_round1(x)

In [10]:
data_clean = pd.DataFrame(grouped_extract.extract.apply(round1))
data_clean.head()

,extract
0,saintnazaire french pronunciation breton sant...
1,gadsby is a novel by ernest vincent wright wr...
2,bombshell is a american precode romantic come...
3,patrick michael shanahan born june is an ame...
4,west side story is a american romantic musica...


In [12]:
# Let's pickle it for later use
data_clean.to_pickle("data_clean.pkl")

# Document-Term Matrix

In [13]:
# We are going to create a document-term matrix using CountVectorizer, and exclude common English stop words
from sklearn.feature_extraction.text import CountVectorizer

In [14]:
cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(data_clean.extract)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.index
data_dtm

,aardvark,aaron,ab,ababa,abandon,abandoned,abba,abbas,abbasid,abbassia,...,ögedei,āṣifat,īsā,ōban,ḥizb,ḥiṣān,ḥusayn,ḥusnī,ḥusēn,ḵāšuqjī
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [15]:
# Let's pickle it for later use
data_dtm.to_pickle("dtm.pkl")

In [16]:
# Find the top 30 words in each cluster
top_dict = {}
for c in data_dtm.transpose().columns:
    top = data_dtm.transpose()[c].sort_values(ascending=False).head(30)
    top_dict[c]= list(zip(top.index, top.values))

top_dict

{0: [('french', 27),
  ('edward', 25),
  ('battle', 24),
  ('france', 23),
  ('army', 20),
  ('king', 20),
  ('order', 17),
  ('war', 16),
  ('son', 14),
  ('juan', 14),
  ('second', 13),
  ('british', 13),
  ('napoleon', 13),
  ('blücher', 13),
  ('carlos', 12),
  ('english', 12),
  ('england', 12),
  ('members', 11),
  ('years', 11),
  ('waterloo', 11),
  ('military', 11),
  ('henry', 11),
  ('city', 11),
  ('days', 10),
  ('paris', 10),
  ('coalition', 10),
  ('reign', 9),
  ('europe', 9),
  ('world', 9),
  ('toulouse', 9)],
 1: [('championship', 31),
  ('wwe', 31),
  ('united', 31),
  ('american', 22),
  ('scientology', 22),
  ('city', 22),
  ('champion', 19),
  ('wrestling', 19),
  ('professional', 17),
  ('title', 17),
  ('team', 17),
  ('club', 17),
  ('league', 16),
  ('soccer', 15),
  ('church', 14),
  ('world', 14),
  ('creed', 13),
  ('tampa', 13),
  ('later', 13),
  ('north', 13),
  ('states', 13),
  ('wrestler', 13),
  ('year', 13),
  ('tna', 13),
  ('area', 12),
  ('novel

In [17]:
# Look at the most common top words --> add them to the stop word list
from collections import Counter

# Let's first pull out the top 30 words for each cluster
words = []
for cluster in data_dtm.transpose().columns:
    top = [word for (word, count) in top_dict[cluster]]
    for t in top:
        words.append(t)
        
words

['french',
 'edward',
 'battle',
 'france',
 'army',
 'king',
 'order',
 'war',
 'son',
 'juan',
 'second',
 'british',
 'napoleon',
 'blücher',
 'carlos',
 'english',
 'england',
 'members',
 'years',
 'waterloo',
 'military',
 'henry',
 'city',
 'days',
 'paris',
 'coalition',
 'reign',
 'europe',
 'world',
 'toulouse',
 'championship',
 'wwe',
 'united',
 'american',
 'scientology',
 'city',
 'champion',
 'wrestling',
 'professional',
 'title',
 'team',
 'club',
 'league',
 'soccer',
 'church',
 'world',
 'creed',
 'tampa',
 'later',
 'north',
 'states',
 'wrestler',
 'year',
 'tna',
 'area',
 'novel',
 'born',
 'new',
 'known',
 'death',
 'vanderbilt',
 'american',
 'new',
 'family',
 'york',
 'cornelius',
 'known',
 'born',
 'member',
 'jeopardy',
 'century',
 'whitney',
 'city',
 'gloria',
 'states',
 'prominent',
 'william',
 'july',
 'united',
 'film',
 'comedy',
 'trial',
 'john',
 'january',
 'best',
 'jennings',
 'furness',
 'world',
 'october',
 'built',
 'secretary',
 'sha

In [18]:
# If more than half of the cluster have it as a top word, exclude it from the list
add_stop_words = [word for word, count in Counter(words).most_common() if count > len(df)]
add_stop_words

[]

In [19]:
# Let's update our document-term matrix with the new list of stop words
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer

# Read in cleaned data
data_clean = pd.read_pickle('data_clean.pkl')

# Add new stop words
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

# Recreate document-term matrix
cv = CountVectorizer(stop_words=stop_words)
data_cv = cv.fit_transform(data_clean.extract)
data_stop = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_stop.index = data_clean.index

# Pickle it for later use
import pickle
pickle.dump(cv, open("cv_stop.pkl", "wb"))
data_stop.to_pickle("dtm_stop.pkl")

# Topic Modeling 

### First Attempt : all the words

In [20]:
# Import the necessary modules for LDA with gensim
from gensim import matutils, models
import scipy.sparse

In [21]:
# One of the required inputs is a term-document matrix
tdm = data_dtm.transpose()
tdm.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
aardvark,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0
aaron,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ab,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ababa,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
abandon,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [22]:
# We're going to put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [23]:
# Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
cv = pickle.load(open("cv_stop.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

In [24]:
# Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term),
# we need to specify two other parameters as well - the number of topics and the number of passes
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=5, passes=40)
lda.print_topics()

[(0,
  '0.006*"air" + 0.006*"aircraft" + 0.005*"united" + 0.005*"game" + 0.004*"states" + 0.004*"american" + 0.003*"force" + 0.003*"games" + 0.003*"military" + 0.003*"series"'),
 (1,
  '0.010*"series" + 0.005*"television" + 0.004*"born" + 0.004*"american" + 0.004*"film" + 0.004*"known" + 0.003*"dc" + 0.003*"comics" + 0.003*"boys" + 0.002*"new"'),
 (2,
  '0.007*"american" + 0.007*"film" + 0.005*"born" + 0.004*"united" + 0.004*"series" + 0.003*"states" + 0.003*"new" + 0.003*"known" + 0.003*"best" + 0.003*"vanderbilt"'),
 (3,
  '0.005*"united" + 0.004*"aircraft" + 0.004*"air" + 0.003*"states" + 0.003*"saudi" + 0.003*"iran" + 0.003*"war" + 0.003*"british" + 0.002*"government" + 0.002*"new"'),
 (4,
  '0.010*"dynasty" + 0.008*"chinese" + 0.006*"emperor" + 0.006*"tang" + 0.005*"han" + 0.004*"nanyue" + 0.004*"li" + 0.003*"china" + 0.003*"sui" + 0.003*"jade"')]

### Second Attempt : nouns only

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

In [54]:
# Let's create a function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

In [55]:
# Read in the cleaned data, before the CountVectorizer step
data_clean = pd.read_pickle('data_clean.pkl')
data_clean

,extract
0,saintnazaire french pronunciation breton sant...
1,gadsby is a novel by ernest vincent wright wr...
2,bombshell is a american precode romantic come...
3,patrick michael shanahan born june is an ame...
4,west side story is a american romantic musica...
5,the mikoyan russian nato reporting name ful...
6,the witches is a dark fantasy comedy film dir...
7,samara weaving born february is an australia...
8,elite is a space trading video game it was wr...
9,thirtysix views of mount fuji fugaku sanjūrok...


In [56]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns = pd.DataFrame(data_clean.extract.apply(nouns))
data_nouns

,extract
0,french neñseir western traditional major right...
1,novel ernest fictional little noticed constrai...
2,american romantic jean morgan c mary franchot ...
3,american retired general white mark last june ...
4,west american romantic musical robert broadway...
5,mikoyan russian russian united further first s...
6,dark anjelica rowan same original evil ordinar...
7,born australian further indi australian ready ...
8,ian acorn september revolutionary contemporary...
9,thirtysix fugaku japanese ukiyoe mount differe...


In [33]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words=stop_words)
data_cvn = cvn.fit_transform(data_nouns.extract)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names())
data_dtmn.index = data_nouns.index
data_dtmn

,aardvark,aaron,ab,ababa,abba,abbas,abbasid,abbassia,abbey,abbreviation,...,éomer,ögedei,āṣifat,īsā,ōban,ḥiṣān,ḥusayn,ḥusnī,ḥusēn,ḵāšuqjī
0,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [34]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [35]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.005*"president" + 0.005*"war" + 0.004*"states" + 0.004*"government" + 0.003*"world" + 0.003*"june" + 0.003*"city" + 0.003*"france" + 0.003*"egypt" + 0.003*"dynasty"'),
 (1,
  '0.010*"film" + 0.008*"series" + 0.005*"states" + 0.005*"television" + 0.005*"aircraft" + 0.005*"air" + 0.004*"films" + 0.003*"award" + 0.003*"family" + 0.003*"album"')]

In [37]:
# Let's start with more topics
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.005*"city" + 0.005*"championship" + 0.005*"war" + 0.004*"france" + 0.004*"battle" + 0.004*"army" + 0.004*"world" + 0.003*"secretary" + 0.003*"years" + 0.003*"year"'),
 (1,
  '0.015*"film" + 0.011*"series" + 0.007*"television" + 0.005*"films" + 0.005*"states" + 0.005*"award" + 0.005*"actress" + 0.005*"album" + 0.004*"actor" + 0.004*"role"'),
 (2,
  '0.008*"game" + 0.006*"games" + 0.005*"vanderbilt" + 0.005*"family" + 0.005*"century" + 0.004*"dynasty" + 0.004*"emperor" + 0.004*"series" + 0.004*"york" + 0.003*"tang"'),
 (3,
  '0.012*"aircraft" + 0.012*"air" + 0.008*"president" + 0.008*"states" + 0.006*"war" + 0.005*"iran" + 0.005*"government" + 0.005*"force" + 0.004*"egypt" + 0.004*"systems"')]

### Third Attempt : nouns and adjectives only

In [38]:
# Let's create a function to pull out nouns from a string of text
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [40]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns_adj = pd.DataFrame(data_clean.extract.apply(nouns_adj))
data_nouns_adj

,extract
0,saintnazaire french pronunciation breton neñse...
1,gadsby novel ernest vincent wright lipogram wo...
2,bombshell american precode romantic comedydram...
3,patrick michael june american government offic...
4,west side story american romantic musical dram...
5,mikoyan russian nato reporting name fulcrumf r...
6,witches dark fantasy comedy film nicolas roeg ...
7,samara born february australian actress model ...
8,elite space trading video game david braben ia...
9,thirtysix views mount fuji fugaku sanjūrokkei ...


In [41]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvna = CountVectorizer(stop_words=stop_words, max_df=.8)
data_cvna = cvna.fit_transform(data_nouns_adj.extract)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names())
data_dtmna.index = data_nouns_adj.index
data_dtmna

,aardvark,aaron,ab,ababa,abba,abbas,abbasid,abbassia,abbey,abbreviation,...,ögedei,āṣifat,īsā,ōban,ḥizb,ḥiṣān,ḥusayn,ḥusnī,ḥusēn,ḵāšuqjī
0,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [42]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [43]:
# Let's start with 2 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.008*"film" + 0.004*"television" + 0.003*"films" + 0.003*"family" + 0.003*"vanderbilt" + 0.003*"award" + 0.003*"game" + 0.003*"actress" + 0.002*"actor" + 0.002*"best"'),
 (1,
  '0.004*"president" + 0.004*"air" + 0.004*"military" + 0.003*"aircraft" + 0.003*"iran" + 0.003*"area" + 0.003*"egyptian" + 0.003*"government" + 0.002*"film" + 0.002*"egypt"')]

In [45]:
# Let's start with 2 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.007*"vanderbilt" + 0.006*"game" + 0.005*"film" + 0.005*"family" + 0.004*"games" + 0.003*"york" + 0.003*"right" + 0.003*"species" + 0.002*"whales" + 0.002*"video"'),
 (1,
  '0.009*"album" + 0.004*"music" + 0.004*"band" + 0.003*"sexual" + 0.003*"studio" + 0.003*"albums" + 0.003*"security" + 0.003*"clarkson" + 0.003*"women" + 0.003*"guitarist"'),
 (2,
  '0.013*"film" + 0.006*"television" + 0.005*"films" + 0.004*"actor" + 0.004*"award" + 0.004*"actress" + 0.003*"roles" + 0.003*"awards" + 0.003*"career" + 0.003*"best"'),
 (3,
  '0.007*"air" + 0.006*"aircraft" + 0.005*"military" + 0.005*"president" + 0.003*"government" + 0.003*"iran" + 0.003*"french" + 0.003*"egyptian" + 0.003*"force" + 0.003*"area"')]

In [65]:
# Let's start with 2 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=7, id2word=id2wordna, passes=80)
ldana.print_topics()

[(0,
  '0.007*"saudi" + 0.006*"iran" + 0.004*"government" + 0.004*"property" + 0.004*"iranian" + 0.004*"president" + 0.004*"sanctions" + 0.003*"prints" + 0.003*"fsln" + 0.003*"khashoggi"'),
 (1,
  '0.008*"vanderbilt" + 0.007*"album" + 0.006*"film" + 0.004*"music" + 0.004*"york" + 0.004*"award" + 0.003*"family" + 0.003*"albums" + 0.003*"actor" + 0.003*"band"'),
 (2,
  '0.009*"film" + 0.006*"species" + 0.006*"right" + 0.006*"whales" + 0.005*"love" + 0.004*"whale" + 0.004*"pacific" + 0.003*"north" + 0.003*"parakeet" + 0.003*"women"'),
 (3,
  '0.005*"championship" + 0.005*"french" + 0.004*"france" + 0.004*"battle" + 0.004*"wwe" + 0.003*"army" + 0.003*"title" + 0.003*"scientology" + 0.003*"champion" + 0.003*"death"'),
 (4,
  '0.011*"film" + 0.009*"aircraft" + 0.008*"air" + 0.006*"television" + 0.005*"films" + 0.004*"actress" + 0.004*"force" + 0.003*"actor" + 0.003*"roles" + 0.003*"systems"'),
 (5,
  '0.009*"egyptian" + 0.009*"president" + 0.009*"game" + 0.008*"egypt" + 0.006*"games" + 0.006

# Identify topics in each class

In [77]:
# Let's take a look at which topics each transcript contains
corpus_transformed = ldana[corpusna]
#list(zip([a for [(a,b)] in corpus_transformed], data_dtm.index))
try:
    listt = list(zip(corpus_transformed, data_dtm.index))
except IndexError:
    pass

In [78]:
for i in range(0, len(listt)):
    list_of_topics = []
    for j in range(0, len(listt[i][0])):
        list_of_topics.append(listt[i][0][j][0])
    print('Class:' , listt[i][1], 'Topic(s):', list_of_topics)

Class: 0 Topic(s): [3]
Class: 1 Topic(s): [3]
Class: 2 Topic(s): [1]
Class: 3 Topic(s): [4]
Class: 4 Topic(s): [4]
Class: 5 Topic(s): [4]
Class: 6 Topic(s): [2]
Class: 7 Topic(s): [4]
Class: 8 Topic(s): [5]
Class: 9 Topic(s): [0]
Class: 10 Topic(s): [1]
Class: 11 Topic(s): [0]
Class: 12 Topic(s): [4]
Class: 13 Topic(s): [6]
Class: 14 Topic(s): [4]
Class: 15 Topic(s): [4]
Class: 16 Topic(s): [5]
Class: 17 Topic(s): [1]
